In [1]:
from nltk.corpus import stopwords
import nltk 
import numpy as np
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
import pandas as pd

# read json into a dataframe
df_idf=pd.read_csv("/content/drive/MyDrive/ic/Rows_ic/SAIDA_011_add_rows_2.csv", sep='\t')

# print schema
print("Schema:\n\n",df_idf.dtypes)
print("Number of questions,columns=",df_idf.shape)

Schema:

 label               int64
cod_label          object
url                object
caracteristicas    object
dtype: object
Number of questions,columns= (109, 4)


In [4]:
import re
def pre_process(text):
    
    # lowercase
    text=text.lower()
    
    #remove tags
    text=re.sub("</?.*?>"," <> ",text)
    
    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    
    return text

df_idf['text'] = df_idf['caracteristicas']
df_idf['text'] = df_idf['text'].apply(lambda x:pre_process(x))

#show the first 'text'
df_idf['text'][100]

' suplementos diário oficial prefeitura municipal campinas início diário oficial suplementos suplementos nesta seção encontrará publicações referentes suplementos diário oficial selecione menu abaixo ano preferência ano ano ano ano ano ano ano ano ano ano ano ano ano ano ano ano ano ano ano ano ano ano ano ano ano ano ano avenida anchieta nº campinas sp cep pabx cnpj prefeitura municipal campinas todos direitos reservados início sobre campinas ouvidoria assessoria imprensa galeria fotos contatos pabx ouvidoria atendimento cidadão atendimento tributário sac mail sac campinas sp gov br subprefeituras subprefeitura barão geraldo telefones rua luiz vicentin centro distrito barão geraldo subprefeitura sousas telefones praça sebastião centro sousas subprefeitura nova aparecida telefones avenida dom agnelo rossi vila padre anchieta subprefeitura joaquim egídio telefone rua josé ignácio centro joaquim egídio subprefeitura campo grande telefone rua natale bertucci parque valença subprefeitura o

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
import re

stopwords_list = stopwords.words("portuguese")


#get the text column 
docs=df_idf['text'].tolist()

#create a vocabulary of words, 
#ignore words that appear in 85% of documents, 
#eliminate stop words
cv=CountVectorizer(max_df=0.85,stop_words=stopwords_list)
word_count_vector=cv.fit_transform(docs)

In [6]:
word_count_vector.shape

(109, 8245)

In [7]:
cv=CountVectorizer(max_df=0.85,stop_words=stopwords_list,max_features=10000)
word_count_vector=cv.fit_transform(docs)
word_count_vector.shape

(109, 8245)

In [8]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer()

In [9]:
tfidf_transformer.idf_

array([5.00733319, 5.00733319, 5.00733319, ..., 4.314186  , 3.13553101,
       3.62103882])

In [10]:
Y = df_idf['label'].values
Y

array([1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 4, 4])

In [11]:
X_er = df_idf['text'][:5]
Y_er = Y[:5]

X_ne = df_idf['text'][6:105]
Y_ne = Y[6:105]

X_di = df_idf['text'][105:107]
Y_di = Y[105:107]

X_so = df_idf['text'][-2:]
Y_so = Y[-2:]

# print('ne', len(ne), 'di', len(di),'so', len(so))

print(X_so)

107     orçamento publicação dom diãrio oficial muníc...
108     serviço serviços campinas palma mão preencha ...
Name: text, dtype: object


In [12]:
#Seleção de URLs
Y_teste = [0, 1, 0, 0, 0]

X_teste = list(X_ne[6:7])
X_teste.extend(X_di[:1])
X_teste.extend(X_ne[41:44])



print( len(Y_teste), len(X_teste))


5 5


In [13]:
df_test = pd.DataFrame(X_teste, columns =['caracteristicas'])

In [14]:
# read test docs into a dataframe and concatenate title and body
#df_test=pd.read_csv("/content/drive/MyDrive/ic/Rows_ic/SAIDA_011_add_rows_2.csv", sep='\t')
df_test['text'] = df_test['caracteristicas']
df_test['text'] =df_test['text'].apply(lambda x:pre_process(x))

# get test docs into a list
docs_test=df_test['text'].tolist()
docs_carac=df_test['caracteristicas'].tolist()

In [15]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []

    for idx, score in sorted_items:
        fname = feature_names[idx]
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

In [16]:
# you only needs to do this once
feature_names=cv.get_feature_names()

# get the document that we want to extract keywords from
doc=docs_test[1]

#generate tf-idf for the given document
tf_idf_vector=tfidf_transformer.transform(cv.transform([doc]))

#sort the tf-idf vectors by descending order of scores
sorted_items=sort_coo(tf_idf_vector.tocoo())

#extract only the top n; n here is 10
keywords=extract_topn_from_vector(feature_names,sorted_items,15)

# now print the results
print("\n=====Caracteristicas=====")
print(docs_carac[1])
print("\n===Keywords===")
for k in keywords:
    print(k,keywords[k])


=====Caracteristicas=====
 contratos secretaria justiça governo prefeitura municipal campinas estrutura biblioteca jurídica procon procuradoria geral município início secretaria justiça contratos contratos pesquisar processo administrativo n órgão solicitante contratada n cnpj cpf n objeto opções pesquisa clique aqui exibir ocultar modalidade termo contrato n valor prazo assinatura contrato assinatura contrato modalidade termo contrato n valor prazo assinatura contrato assinatura contrato pesquisar processo administrativo n º interessado secretaria municipal saúde modalidade ata termo contrato n contratada kimenz equipamentos limitada epp cnpj cpf objeto registro preços materiais laboratório kits diagnóstico sífilis valor prazo meses assinatura ver contrato processo administrativo n º interessado secretaria municipal transportes modalidade convênio termo contrato n contratada empresa municipal desenvolvimento campinas s emdec cnpj cpf objeto execução atividades planejamento gerenciame

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [17]:
#idx=len(X_teste)-1
#keywords=get_keywords(idx)
#print_results(idx,keywords)

In [18]:
#generate tf-idf for all documents in your list. docs_test has 500 documents
tf_idf_vector=tfidf_transformer.transform(cv.transform(docs_test))

results=[]
for i in range(tf_idf_vector.shape[0]):
    
    # get vector for a single document
    curr_vector=tf_idf_vector[i]
    
    #sort the tf-idf vector by descending order of scores
    sorted_items=sort_coo(curr_vector.tocoo())

    #extract only the top n; n here is 10
    keywords=extract_topn_from_vector(feature_names,sorted_items,15)
    
    
    results.append(keywords)

df=pd.DataFrame(zip(docs_test,results),columns=['doc','keywords'])
df

,doc,keywords
0,imprensa governo prefeitura municipal campina...,"{'telefone': 0.62, 'imprensa': 0.45, 'assessor..."
1,contratos secretaria justiça governo prefeitu...,"{'contrato': 0.477, 'contratada': 0.263, 'moda..."
2,portal serviços prefeitura municipal campinas...,"{'cultural': 0.185, 'iptu': 0.185, 'saúde': 0...."
3,diário oficial prefeitura municipal campinas ...,"{'edição': 0.335, 'subprefeitura': 0.242, 'diá..."
4,refis campinas refis campinas adesão current ...,"{'refis': 0.377, 'pessoa': 0.224, 'parcelas': ..."


In [19]:
def pre_process2(text):
    
    # lowercase
    text=text.lower()
  
    
    return text


In [20]:
keywords_divulg =pd.read_csv("/content/drive/MyDrive/ic/keywords_divulg.csv", sep='\t')

keywords_divulg['text'] = keywords_divulg['keywords'].apply(lambda x:pre_process2(x))

keywords_divulg = keywords_divulg.reset_index()

In [21]:
keywords_divulg

,index,keywords,text
0,0,"['CPF', 'RG', 'reservista', 'Título de eleitor...","['cpf', 'rg', 'reservista', 'título de eleitor..."


In [22]:
for index, row in keywords_divulg.iterrows():
    print(row['text'])

['cpf', 'rg', 'reservista', 'título de eleitor', 'certidão casamento', 'no convênio médico', 'no convênio odontológico', 'cnh', 'pis', 'ctps', 'nome', 'data de nascimento', 'sexo', 'estado civil','naturalidade', 'nacionalidade', 'etnia', 'nome do cônjuge', 'nome do pai', 'nome da mãe', 'nome dos filhos', 'pensionista', 'conta', 'agência', 'banco', 'conta fgts', 'tipo de conta', 'salário base', 'produtividade', 'índice de produtividade', 'variável', 'remuneração variável', 'benefícios', 'carga horária', 'hora extra', 'horário de entrada', 'horário de saída', 'empresa', 'cargo', 'função', 'líder imediato','departamento' , 'convicções religiosa', 'política', 'filosófica', 'contato', 'telefone particular', 'telefone corporativo', 'e-mail particular', 'e-mail corporativo', 'endereço', 'cep', 'bairro', 'cidade', 'país', 'número', 'complemento', 'pcd', 'cid', 'saúde ocupacional', 'atestados', 'licenças', 'gravidez', 'genético', 'biomédico', 'formação acadêmica', 'experiências anteriores', 'fi

In [23]:
if 'cpf' in results[1]:
    print("Element Found in Tuple")
else:
    print("Element not Found in Tuple")   

Element Found in Tuple


In [24]:
Row_list =[]
  
# Iterate over each row
for index, rows in keywords_divulg.iterrows():
    # Create list for the current row
    my_list =[rows.text]
      
    # append the list to the final list
    Row_list.append(my_list)
  
# Print the list
print(Row_list)

[["['cpf', 'rg', 'reservista', 'título de eleitor', 'certidão casamento', 'no convênio médico', 'no convênio odontológico', 'cnh', 'pis', 'ctps', 'nome', 'data de nascimento', 'sexo', 'estado civil','naturalidade', 'nacionalidade', 'etnia', 'nome do cônjuge', 'nome do pai', 'nome da mãe', 'nome dos filhos', 'pensionista', 'conta', 'agência', 'banco', 'conta fgts', 'tipo de conta', 'salário base', 'produtividade', 'índice de produtividade', 'variável', 'remuneração variável', 'benefícios', 'carga horária', 'hora extra', 'horário de entrada', 'horário de saída', 'empresa', 'cargo', 'função', 'líder imediato','departamento' , 'convicções religiosa', 'política', 'filosófica', 'contato', 'telefone particular', 'telefone corporativo', 'e-mail particular', 'e-mail corporativo', 'endereço', 'cep', 'bairro', 'cidade', 'país', 'número', 'complemento', 'pcd', 'cid', 'saúde ocupacional', 'atestados', 'licenças', 'gravidez', 'genético', 'biomédico', 'formação acadêmica', 'experiências anteriores', 

In [25]:
lista_divulg = []

lista_divulg.append(keywords_divulg['text'])

In [26]:
print(lista_divulg)

[0    ['cpf', 'rg', 'reservista', 'título de eleitor...
Name: text, dtype: object]


In [27]:
for index, row in df.iterrows():
    if 'cpf' in results[index]:
        print("Element Found in Tuple")
    else:
        print("Element not Found in Tuple")   

Element not Found in Tuple
Element Found in Tuple
Element not Found in Tuple
Element not Found in Tuple
Element not Found in Tuple


In [28]:
print(results)

[{'telefone': 0.62, 'imprensa': 0.45, 'assessoria': 0.418, 'secretaria': 0.392, 'subprefeitura': 0.063, 'cohab': 0.054, 'avenida': 0.053, 'assuntos': 0.047, 'pabx': 0.043, 'acesse': 0.043, 'anchieta': 0.042, 'desenvolvimento': 0.04, 'sp': 0.04, 'posteriores': 0.039, 'mistas': 0.039}, {'contrato': 0.477, 'contratada': 0.263, 'modalidade': 0.256, 'termo': 0.254, 'assinatura': 0.223, 'prazo': 0.209, 'objeto': 0.2, 'interessado': 0.199, 'meses': 0.19, 'ver': 0.187, 'cpf': 0.184, 'administrativo': 0.184, 'preços': 0.177, 'valor': 0.17, 'processo': 0.152}, {'cultural': 0.185, 'iptu': 0.185, 'saúde': 0.182, 'ficha': 0.179, 'requerimento': 0.161, 'concurso': 0.141, 'certidão': 0.134, 'consulta': 0.132, 'hospital': 0.13, 'patrimônio': 0.123, 'medicamentos': 0.123, 'sanasa': 0.118, 'emprego': 0.115, 'calendário': 0.115, 'educação': 0.108}, {'edição': 0.335, 'subprefeitura': 0.242, 'diário': 0.229, 'oficial': 0.218, 'avenida': 0.201, 'outubro': 0.161, 'centro': 0.155, 'zink': 0.137, 'pesquise': 0

In [29]:
print(lista_divulg)

[0    ['cpf', 'rg', 'reservista', 'título de eleitor...
Name: text, dtype: object]


In [30]:
list_divulg = ['cpf', 'rg', 'reservista', 'título de eleitor', 'certidão casamento', 'no convênio médico', 'no convênio odontológico', 'cnh', 'pis', 'ctps', 'nome', 'data de nascimento', 'sexo', 'estado civil','naturalidade', 'nacionalidade', 'etnia', 'nome do cônjuge', 'nome do pai', 'nome da mãe', 'nome dos filhos', 'pensionista', 'conta', 'agência', 'banco', 'conta fgts', 'tipo de conta', 'salário base', 'produtividade', 'índice de produtividade', 'variável', 'remuneração variável', 'benefícios', 'carga horária', 'hora extra', 'horário de entrada', 'horário de saída', 'empresa', 'cargo', 'função', 'líder imediato','departamento' , 'convicções religiosa', 'política', 'filosófica', 'contato', 'telefone particular', 'telefone corporativo', 'e-mail particular', 'e-mail corporativo', 'endereço', 'cep', 'bairro', 'cidade', 'país', 'número', 'complemento', 'pcd', 'cid', 'saúde ocupacional', 'atestados', 'licenças', 'gravidez', 'genético', 'biomédico', 'formação acadêmica', 'experiências anteriores', 'filiação', 'oposição assistencial', 'regime trabalhista', 'data admissão', 'data desligamento', 'dados de uso cookies', 'ip', 'localização']

Lista colocada na mão pois transformar de um df complica demais 

In [31]:
qnt = len(X_teste)-1

resultado = []
for x in range(len(X_teste)):
  resultado.append(0)

In [32]:
for index, row in df.iterrows():
  for i in list_divulg:
    if i in results[index]:
        resultado[index] = 1


In [33]:
resultado


[0, 1, 0, 0, 0]

In [34]:
Y_teste

[0, 1, 0, 0, 0]

In [35]:
resultado = np.array(resultado)

In [36]:
def fit_and_predict(result, Y_teste):

	acertos = result == Y_teste

	total_de_acertos = sum(acertos)
	total_de_elementos = len(Y_teste)

	taxa_de_acerto = 100.0 * total_de_acertos / total_de_elementos

	msg = "Taxa de acerto do algoritmo : {0}".format(taxa_de_acerto)

	print(msg)
	return taxa_de_acerto

In [37]:
resultadoTfidf = fit_and_predict(resultado , Y_teste)

Taxa de acerto do algoritmo : 100.0
